<a href="https://colab.research.google.com/github/muratali016/Data-Science-and-ML/blob/main/Finding_area_from_selected_irregular_object_make_predictionswith_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update && apt install -y openslide-tools
!pip install openslide-python

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 88.7 kB in 4s (24.7 kB/s)
Reading package lists... Done
Buildin

In [ ]:
import cv2
import ntpath
import openslide
import numpy as np
from skimage.filters import threshold_otsu
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def FindOverlap(FrameDim, PatchDim, OverlapCoef):
    OverlapVal = 0
    PatchNum = math.ceil((FrameDim + PatchDim - 2 * OverlapCoef * PatchDim) / (2 * PatchDim - 2 * OverlapCoef * PatchDim))
    if PatchNum < 1:
        OverlapVal = 0
    elif PatchNum == 1:
        OverlapVal = 2 * PatchDim - FrameDim
    else:
        GapVal = ((FrameDim - (PatchNum * PatchDim)) / (PatchNum - 1))
        OverlapVal = (PatchDim - GapVal) / 2
        
    return OverlapVal

def CropFrame2Rects(FrameW, FrameH, PatchW, PatchH, OverlapCoefW, OverlapCoefH):
    Res = []
    
    OverlapValW = FindOverlap(FrameW, PatchW, OverlapCoefW)
    OverlapValH = FindOverlap(FrameH, PatchH, OverlapCoefH)
    
    i = 0
    while (i <= (FrameH - PatchH)):
        y = i
        
        j = 0
        while(j <= (FrameW - PatchW)):
            x = j
            y = i
            Res.append([x, y, PatchW, PatchH])
            
            j += PatchW - OverlapValW
            
        i += PatchH - OverlapValH
        
    return Res

def get_annot_info(txt_path):
    f = open(txt_path ,'r')
    lines = f.readlines()
    
    annotations = []
    for line in lines:
        x, y, w, h = np.int_(np.float_(line.strip().split("//")[1].strip().split()))
        annotations.append([x, y, w, h])
    return annotations

def get_mask(image):
    
    image = 255 - image
    kernel = np.ones((5,5),np.uint8)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    
    kernel2 = np.ones((30,30), np.uint8)
    kernel2 = kernel2/np.sum(kernel2)
    image_lp = cv2.filter2D(image,-1,kernel2)

    thresh = threshold_otsu(image_lp)
    mask = image_lp > thresh
    mask = (mask+0)*255
    
    return mask

In [ ]:
from PIL import Image
import os, os.path

imgs = []
path = "/content/drive/MyDrive/TIL data/ctf-hasta"
valid_images = [".tif"]
def find_images(path):
  list_image=[]
  list_text=[]
  for f in os.listdir(path):
      ext = os.path.splitext(f)[1]
      if ext.lower() not in valid_images:
          list_text.append(f)
          continue 
      
      list_image.append(f)    
  return list_image,list_text
list_image,list_text=find_images(path)
list_image,list_text

(['CTP 1 14562 - 1.tif',
  'CTP 1 14776 - 1.tif',
  'CTP 1 14893 - 1.tif',
  'CTP 1 177 - 1.tif',
  'CTP 1 1910 - 1.tif',
  'CTP 1 2317 - 1.tif',
  'CTP 1 3137 - 1.tif',
  'CTP 1 3478 - 1.tif',
  'CTP 1 4465 - 1.tif',
  'CTP 1 4585 - 1.tif',
  'CTP 1 5315 - 1.tif',
  'CTP 1 5632 - 1.tif',
  'CTP 1 6174 - 1.tif',
  'CTP 1 6558 - 1.tif',
  'CTP 1 6839 - 1.tif',
  'CTP 1 6976 - 1.tif',
  'CTP 1 6976 - 2.tif',
  'CTP 1 6977 - 1.tif',
  'CTP 1 7101 - 1.tif',
  'CTP 1 7194 - 1.tif',
  'CTP 1 7246 - 1.tif',
  'CTP 1 7258 - 1.tif',
  'CTP 1 7296 - 1.tif',
  'CTP 1 7314 - 1.tif',
  'CTP 1 8059 - 1.tif',
  'CTP 1 8290 - 1.tif',
  'CTP 1 8631 - 1.tif',
  'CTP 1 8696 - 1.tif',
  'CTP 1 8696 - 2.tif',
  'CTP 1 8869 - 1.tif',
  'CTP 1 8957 - 1.tif',
  'CTP 1 9125 - 1.tif',
  'CTP 1 9616 - 1.tif',
  'CTP 1 9896 - 1.tif',
  'CTP 1 10086 - 1.tif',
  'CTP 1 10143 - 1.tif',
  'CTP 1 10389 - 1.tif',
  'CTP 1 11723 - 1.tif',
  'CTP 1 12086 - 1.tif',
  'CTP 1 12802 - 1.tif',
  'CTP 1 13512 - 1.tif',
  'CTP 

In [ ]:
output_folder = "/content/drive/MyDrive/finding_areas"
patch_size = 1000
def path (list_image,list_text): 
  images=[]
  for i in range(len(list_image)):
    a="/content/drive/MyDrive/TIL data/ctf-hasta/"+list_image[i]
    images.append(a)

  texts=[]  
  for y in range(len(list_text)):
    b="/content/drive/MyDrive/TIL data/ctf-hasta/"+list_text[y]
    texts.append(b)
  return images,texts
images,texts=path(list_image,list_text) 
images,texts

(['/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 14562 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 14776 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 14893 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 177 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 1910 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 2317 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 3137 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 3478 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 4465 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 4585 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 5315 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 5632 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 6174 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 6558 - 1.tif',
  '/content/drive/MyDrive/TIL data/ctf-hasta/CTP 1 6839 - 1.tif',
  '/cont

In [ ]:
image_namess=[]
for i in range(len(images)):
  # Slide
  head, tail = ntpath.split(images[i])
  slide_name = tail.split(".")[0]
  image_namess.append(slide_name)   
  annotations = get_annot_info(texts[i])              
  slide = openslide.OpenSlide(images[i])
  s_w, s_h = slide.dimensions

  for idx, level in enumerate(slide.level_downsamples):       
      if int(level) == 16 or int(level) == 32:
          level_mask = idx
              
  w1, h1 = slide.level_dimensions[level_mask]
  low_res_slide = np.array(slide.read_region((0,0), level_mask, (w1, h1)).convert('RGB').convert('L'))
  # Mask
  mask = get_mask(low_res_slide)
  cv2.imwrite(output_folder + "/" + "{}.bmp".format(slide_name), mask)

In [ ]:
from google.colab.patches import cv2_imshow
path="/content/drive/MyDrive/finding_areas"

def load_images_from_folder(folder):
    total_areass=[]
    images = []
    matching_images=[] 
    for filename in os.listdir(folder):
      matching_images.append(filename)
      img = cv2.imread(os.path.join(folder,filename))
      img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      imgBlur = cv2.GaussianBlur(img, (7, 7), 1) 
      imgCanny = cv2.Canny(imgBlur, 255, 195)

      kernel = np.ones((2, 2))
      imgDil = cv2.dilate(imgCanny, kernel, iterations = 3)
      imgThre = cv2.erode(imgDil, kernel, iterations = 3)
      
      if img is not None:
        images.append(img)
      contours, hierarchy = cv2.findContours(imgThre, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    
      
      contours_areas=[]
      for cnt in contours:       
          area = cv2.contourArea(cnt)
          contours_areas.append(area)                                    
      total_area=0
      for ele in range(len(contours_areas)):
        total_area = total_area + contours_areas[ele]
      total_areass.append(total_area)
          
    return total_areass,matching_images       
    
total_areas,matching_images=load_images_from_folder(path)

In [ ]:
len(total_areas),len(matching_images),print(total_areas,-1)

[2979295.0, 126059.5, 995431.0, 314451.5, 83395.5, 188899.0, 166285.5, 354850.5, 245122.5, 158702.5, 1922.5, 552212.0, 375959.0, 48251.0, 42174.5, 2437561.0, 3236213.5, 166362.0, 1729620.0, 342307.5, 99298.5, 176306.0, 88171.0, 31977.5, 293597.0, 178146.0, 417382.5, 158271.5, 1722538.5, 177807.0, 176134.5, 142660.5, 109186.5, 478487.0, 182103.0, 2502.5, 89675.5, 458295.0, 2547534.0, 426431.5, 183293.5, 610868.5, 232167.5, 181245.0] -1


(44, 44, None)

In [ ]:
 import pandas as pd
 data=pd.DataFrame({"image_name":matching_images,"image_area":total_areas})
 data

,image_name,image_area
0,CTP 1 14562 - 1.bmp,2979295.0
1,CTP 1 14776 - 1.bmp,126059.5
2,CTP 1 14893 - 1.bmp,995431.0
3,CTP 1 177 - 1.bmp,314451.5
4,CTP 1 1910 - 1.bmp,83395.5
5,CTP 1 2317 - 1.bmp,188899.0
6,CTP 1 3137 - 1.bmp,166285.5
7,CTP 1 3478 - 1.bmp,354850.5
8,CTP 1 4465 - 1.bmp,245122.5
9,CTP 1 4585 - 1.bmp,158702.5


In [ ]:
patch_count=pd.read_csv("/content/MLLLL.csv")
patch_count["patch_count"]

0      548
1       33
2     2501
3       15
4       56
5       25
6      522
7     1461
8     1592
9       37
10       0
11      23
12    1507
13     418
14    2163
15    2800
16     105
17    1222
18    1663
19    2267
20    1528
21      50
22      11
23    2874
24      36
25      56
26    3014
27    1150
28     110
29     150
30      54
31     119
32      28
33    2445
34      42
35      39
36    1552
37     954
38      24
39    2156
40    1278
41    1245
42    1503
43     339
Name: patch_count, dtype: int64

In [ ]:
 data=pd.DataFrame({"image_name":matching_images,"image_area":total_areas,"patch_count":patch_count["patch_count"]})
 data

,image_name,image_area,patch_count
0,CTP 1 14562 - 1.bmp,2979295.0,548
1,CTP 1 14776 - 1.bmp,126059.5,33
2,CTP 1 14893 - 1.bmp,995431.0,2501
3,CTP 1 177 - 1.bmp,314451.5,15
4,CTP 1 1910 - 1.bmp,83395.5,56
5,CTP 1 2317 - 1.bmp,188899.0,25
6,CTP 1 3137 - 1.bmp,166285.5,522
7,CTP 1 3478 - 1.bmp,354850.5,1461
8,CTP 1 4465 - 1.bmp,245122.5,1592
9,CTP 1 4585 - 1.bmp,158702.5,37


In [ ]:
"""
test_row=43
train_idx=np.arange(data.shape[0])!=test_row
test_idx=np.arange(data.shape[0])==test_row
"""
from sklearn.preprocessing import OneHotEncoder 
 # creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df = pd.DataFrame(enc.fit_transform(data[['image_name']]).toarray())
# merge with main df bridge_df on key values
data = data.join(enc_df)

 

data_train=data.drop("image_name" ,axis=1)
data_test=data["patch_count"]


In [ ]:
data_train=data_train.to_numpy()
data_test=data_test.to_numpy()
 
data_train.T.shape,data_test.T.shape
data_train=data_train
data_test=data_test.T


In [ ]:
data_train.shape,data_test.shape

((44, 46), (44,))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

model=RandomForestRegressor(n_jobs=-1,random_state=42)

X_train,X_test,y_train,y_test=train_test_split(data_train,data_test,test_size=0.2)

model_grids={"n_estimators":np.arange(10,100,10),
"max_depth":[None,3,5,10],
"min_samples_split":np.arange(2,20,2),
"min_samples_leaf":np.arange(1,20,2),
"max_features":[0.5,1,"sqrt","auto"],
}

model=RandomizedSearchCV(model,param_distributions=model_grids,n_iter=120,
cv=2,verbose=True)

model.fit(X_train, y_train)  

Fitting 2 folds for each of 120 candidates, totalling 240 fits


RandomizedSearchCV(cv=2,
                   estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
                   n_iter=120,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'max_features': [0.5, 1, 'sqrt',
                                                         'auto'],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
                   verbose=True)

In [ ]:
model.best_params_

{'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'n_estimators': 90}

In [ ]:
ideal_model=RandomForestRegressor(n_estimators= 10,
 min_samples_split= 2,
 min_samples_leaf=3,
 max_features= 'auto',
 max_depth=10,
 
 random_state=42)
ideal_model.fit(X_train,y_train)

RandomForestRegressor(max_depth=10, min_samples_leaf=3, n_estimators=10,
                      random_state=42)

In [ ]:
from sklearn.metrics import mean_squared_log_error,mean_absolute_error
from sklearn.metrics import r2_score
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds=model.predict(X_test)
    scores={"Training MAE ": mean_absolute_error(y_train,train_preds),
             "Test MAE ": mean_absolute_error(y_test,val_preds),             
            "Training R^2":r2_score( y_train,train_preds),
            "Test R^2 ":r2_score(y_test,val_preds) } 
    return scores 

In [ ]:
scores=show_scores(ideal_model)
scores

{'Test MAE ': 60.67035714285708,
 'Test R^2 ': 0.9858776693065727,
 'Training MAE ': 65.83110090702947,
 'Training R^2': 0.9790490911394887}

In [ ]:
y_preds=ideal_model.predict(X_test)
df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_preds})
df

,Actual,Predicted
0,1461,1490.220000
1,1507,1490.220000
2,2800,2467.191429
3,1278,1287.438095
4,2501,2467.191429
5,105,92.024167
6,522,420.136667
7,36,44.151190
8,33,33.987619


In [ ]:
import pickle
filename = 'ideal_model.sav'
pickle.dump(ideal_model, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
scores=show_scores(loaded_model)
scores 

{'Test MAE ': 60.67035714285708,
 'Test R^2 ': 0.9858776693065727,
 'Training MAE ': 65.83110090702947,
 'Training R^2': 0.9790490911394887}